In [14]:
import pandas as pd
import re
import csv

pd.set_option('display.max_colwidth', None)

def contains_arabic(text):
    arabic_pattern = re.compile('[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+')
    return bool(arabic_pattern.search(text))

def contains_url(text):
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return bool(url_pattern.search(text))

def contains_noise(text):
    # Check for short length
    if len(text.split()) < 4:
        return True

    # Check for repetitive characters
    if re.search(r'(.)\1{2,}', text):
        return True

    # Check for non-Arabic characters (assuming Arabic text is in Unicode range)
    if not re.search(r'[\u0600-\u06FF]', text):
        return True

    # Check for excessive punctuation
    if re.search(r'[!?.]{4,}', text):
        return True

    # Check for excessive numbers
    if re.search(r'\d{5,}', text):
        return True

    # Check for URLs
    if re.search(r'http\S+|www.\S+', text):
        return True

    return False


# Exclude Arabic characters
def check_non_ascii_tweets(df):
    non_ascii_df = df[df['tweet_text'].apply(lambda x: any((ord(char) > 127 and ord(char) < 1536 and not '\u0600' <= char <= '\u06FF') for char in x))]
    return non_ascii_df


def data_exploration(train_df):
  print (train_df.shape)
  class_distribution = train_df['class_label'].value_counts(normalize=True) * 100
  print("Class Distribution:")
  print(class_distribution)

  train_df['text_length'] = train_df['tweet_text'].apply(len)
  missing_values = train_df.isnull().sum()
  print("Missing values:")
  print(missing_values)
  duplicate_rows = train_df.duplicated().sum()
  print(f"Number of duplicate rows: {duplicate_rows}")

  arabic_tweets = train_df[train_df['tweet_text'].apply(contains_arabic)]
  print(f"Number of Arabic tweets: {len(arabic_tweets)}")

  tweets_with_url = train_df[train_df['tweet_text'].apply(contains_url)]
  print(f"Number of tweets with URL: {len(tweets_with_url)}")

  tweets_with_noise = train_df[train_df['tweet_text'].apply(contains_noise)]
  print(f"Number of tweets with noise: {len(tweets_with_noise)}")

  non_ascii_df = check_non_ascii_tweets(train_df)
  print(f"Number of tweets with non-ASCII characters: {len(non_ascii_df)}\n")
  print("Examples:")
  print(non_ascii_df['tweet_text'].head())

  return

def load(filename):
    df = pd.read_csv(filename, sep='\t', encoding='utf-8', names=['tweet_id', 'tweet_url', 'tweet_text', 'class_label'], quoting=csv.QUOTE_NONE, skiprows=1, dtype={'tweet_id': 'Int64'})
    data_exploration(df)
    df.drop(columns=['tweet_url'], inplace=True)
    return df



train_df = load('CT24_checkworthy_arabic/CT24_checkworthy_arabic_train.tsv')
train_df.head()


(7333, 4)
Class Distribution:
class_label
No     69.412246
Yes    30.587754
Name: proportion, dtype: float64
Missing values:
tweet_id       0
tweet_url      0
tweet_text     0
class_label    0
text_length    0
dtype: int64
Number of duplicate rows: 0
Number of Arabic tweets: 7333
Number of tweets with URL: 4845
Number of tweets with noise: 5174
Number of tweets with non-ASCII characters: 126

Examples:
176                                                                                                                                                                                               مكونة من 80 صفحة.. «#ترامب» يعلن «#صفقة_القرن».. وهذه أهم بنودها 👇 https://t.co/WjBh4jzZWh
277                                          صفقة لحل النزاع؟ أم لزيادة السيطرة الإسرائيلية على أرض #فلسطين؟ «#صفقة_القرن» مكونة من مئات البنود مكتوبة في 181 ورقة، وتتوزع بنودها بين الشقين السياسي والاقتصادي. في هذه السلسلة نقدم لكم كل شيء عن «صفقة القرن» 1/11 https://t.co/5XtiFpvWiD
288    🌟ومضة: ▪️كل فصيل 

,tweet_id,tweet_text,class_label,text_length
0,1221949644554587904,وأي خيانة أكبر من خيانة الدين ،، الوطن ،، الهوية ،، العقيدة ،، التاريخ .. عليسة المرأة والتي لا تمتلك مما فات شيئا قالت النار ولا العار يا أمة المليار .. #صفقة_القرن #القدس_عاصمة_فلسطين_الأبدية,No,193
1,1222030473385345024,صباح الخير على فلسطين وقدسها وشعبها ومهجرينها في المخيمات وأسراها ومبعديها ، صباح الخير لكل من يحمل في قلبه حبها وانتمائها 💛🌼 #القدس_عاصمة_فلسطين_الأبديه #تسقط_صفقة_الوهم https://t.co/jGjx9Rq28c,No,194
2,1222035929105338368,ذنبگ أنگ جميله گ يوسف وخانگ العالم گ أاخواته 💔 صباحكم فلسطيني صباح الوطن الجميل #صفقة_القرن_لن_تمر https://t.co/FEHdfHaupP,No,122
3,1222048121145962496,لا يلزمك ان تكون فلسطينياً لتحب فلسطين، حُبها لا يعرف جنسية أو هوية.. حُبها يكون بالفطرة.. شيء يشبه حُبنا لأمهاتنا بلا تفكير! ❤ #صباح_الخير 💛🇵🇸💛 #القدس_عاصمة_فلسطين_الأبدية ⁦✌️⁩ https://t.co/gUPpBAGGPy,No,201
4,1222053294266372096,"#مناصرون رحل البروفسور زيك فريد فوغل . رئيس قسم جراحة الأعصاب في مستشفى مايو كلينك - برلين. من أفضل ٥ جراحين الأعصاب على مستوى العالم . عمل ٤٥٠ عمليه سرطان دماغ في فلسطين ، والاف عمليات العمود الفقري وهو يلفظ انفاسه قال قولوا للفلسطينيين سوف تنتصرون."" #صفقة_القرن_لن_تمر https://t.co/EtK2eJ4K1T",No,294


## Preprocessing



Amb. words:
    أها (Aha)
    آه (Ah)
    يع (Yeah)
    همم (Hm)
    ههه (Haha, similar to "lol" in English)
    هوو (Hoo)

    Remove URLs: remove_urls
    Replace Repeated Special Characters: replace_repeated_characters
    Replace Commas and Double Quotes: replace_commas_and_quotes
    Remove Ambiguous Words: remove_ambiguous_words
    Remove Non-ASCII Characters: remove_non_ascii
    

In [15]:
import regex as re
import numpy as np
from sklearn.preprocessing import LabelEncoder


def remove_urls(text):
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)

    # Fix user
    text = re.sub(r'(@\w+\s*)+', '@<USER> ', text)

    return text

def remove_non_ascii(text):
    # Remove non-ASCII characters except Arabic script
    # Keep Arabic script characters while removing other non-ASCII characters
    text = re.sub(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\x00-\x7F]+', ' ', text)
    return text




def replace_repeated_characters(text):
    # Replace repeated special characters with a single occurrence
    text = re.sub(r'([!?]){2,}', r'\1', text)
    return text

def replace_commas_and_quotes(text):
    # Replace ،، with a single comma ,
    text = text.replace('،،', ',')

    # Replace multiple double quotes with a single occurrence
    text = re.sub(r'"{2,}', '"', text)
    return text

def remove_ambiguous_words(text):
    ambiguous_words = ['أها', 'آه', 'يع', 'ههه', 'همم', 'هوو']

    for word in ambiguous_words:
        # Remove the ambiguous word and fix spaces
        text = re.sub(r'\b{}\b'.format(word), '', text)
        text = re.sub(r'\s+', ' ', text).strip()  # Fix extra spaces

        # Remove comma or punctuation on the left side of the removed word
        text = re.sub(r',?\s*{}'.format(word), '', text)

    return text


# Function to extract hashtags from tweet text
def extract_hashtags(text):
    hashtags = re.findall(r"#(\w+)", text)
    return " ".join(hashtags)

# Function to extract mentions from tweet text
def extract_mentions(text):
    mentions = re.findall(r"@(\w+)", text)
    return " ".join(mentions)


def preprocessing(train_df):
    train_df['hashtags'] = train_df['tweet_text'].apply(extract_hashtags)
    train_df['mentions'] = train_df['tweet_text'].apply(extract_mentions)

    train_df['tweet_text'] = train_df['tweet_text'].apply(remove_urls)
    train_df['tweet_text'] = train_df['tweet_text'].apply(replace_commas_and_quotes)
    train_df['tweet_text'] = train_df['tweet_text'].apply(replace_repeated_characters)
    train_df['tweet_text'] = train_df['tweet_text'].apply(remove_ambiguous_words)
    train_df['tweet_text'] = train_df['tweet_text'].apply(remove_non_ascii)

    # Calculate text length
    train_df['text_length'] = train_df['tweet_text'].apply(len)

    # Categorize text length
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, np.inf]
    labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '100+']
    train_df['text_length_category'] = pd.cut(train_df['text_length'], bins=bins, labels=labels)

    # Label encode class label
    label_encoder = LabelEncoder()
    train_df['class_label_encoded'] = label_encoder.fit_transform(train_df['class_label'])


    train_df = train_df.drop(columns=['text_length'])

    #data_exploration(train_df)
    return train_df

preprocessing(train_df)
train_df.head()


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded
0,1221949644554587904,"وأي خيانة أكبر من خيانة الدين , الوطن , الهوية , العقيدة , التاريخ .. عليسة المرأة والتي لا تمتلك مما فات شيئا قالت النار ولا العار يا أمة المليار .. #صفقة_القرن #القدس_عاصمة_فلسطين_الأبدية",No,189,صفقة_القرن القدس_عاصمة_فلسطين_الأبدية,,100+,0
1,1222030473385345024,صباح الخير على فلسطين وقدسها وشعبها ومهجرينها في المخيمات وأسراها ومبعديها ، صباح الخير لكل من يحمل في قلبه حبها وانتمائها #القدس_عاصمة_فلسطين_الأبديه #تسقط_صفقة_الوهم,No,169,القدس_عاصمة_فلسطين_الأبديه تسقط_صفقة_الوهم,,100+,0
2,1222035929105338368,ذنبگ أنگ جميله گ يوسف وخانگ العالم گ أاخواته صباحكم فلسطيني صباح الوطن الجميل #صفقة_القرن_لن_تمر,No,98,صفقة_القرن_لن_تمر,,91-100,0
3,1222048121145962496,لا يلزمك ان تكون فلسطينياً لتحب فلسطين، حُبها لارف جنسية أو هوية.. حُبها يكون بالفطرة.. شيء يشبه حُبنا لأمهاتنا بلا تفكير! #صباح_الخير #القدس_عاصمة_فلسطين_الأبدية,No,168,صباح_الخير القدس_عاصمة_فلسطين_الأبدية,,100+,0
4,1222053294266372096,"#مناصرون رحل البروفسور زيك فريد فوغل . رئيس قسم جراحة الأعصاب في مستشفى مايو كلينك - برلين. من أفضل ٥ جراحين الأعصاب على مستوى العالم . عمل ٤٥٠ عمليه سرطان دماغ في فلسطين ، والاف عمليات العمود الفقري وهو يلفظ انفاسه قال قولوا للفلسطينيين سوف تنتصرون."" #صفقة_القرن_لن_تمر",No,270,مناصرون صفقة_القرن_لن_تمر,,100+,0


## Feature engineering
  * Add Frequency of Hashtags
  * Sentiment Analysis of Hashtags
  * Topic Modeling with LDA

In [16]:
from textblob import TextBlob
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Function to count frequency of hashtags
def count_hashtags_frequency(text):
    hashtags = re.findall(r"#(\w+)", text)
    return len(hashtags)

# Function to analyze sentiment of hashtags
def analyze_hashtag_sentiment(text):
    hashtags = re.findall(r"#(\w+)", text)
    if hashtags:
        sentiment_scores = [TextBlob(hashtag).sentiment.polarity for hashtag in hashtags]
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
        return avg_sentiment
    else:
        return 0

# Function for topic modeling with LDA
def topic_modeling_with_lda(texts):
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    dtm = vectorizer.fit_transform(texts)

    lda = LatentDirichletAllocation(n_components=5, random_state=42)
    lda.fit(dtm)

    topics = lda.transform(dtm)

    return topics.argmax(axis=1)

def add_additional_features(train_df):
    # Count frequency of hashtags
    train_df['hashtags_frequency'] = train_df['tweet_text'].apply(count_hashtags_frequency)

    # Analyze sentiment of hashtags
    train_df['hashtags_sentiment'] = train_df['tweet_text'].apply(analyze_hashtag_sentiment)

    # Topic modeling with LDA
    train_df['hashtags_topics'] = topic_modeling_with_lda(train_df['hashtags'])

    return

add_additional_features(train_df)
train_df.head(4)

,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,1221949644554587904,"وأي خيانة أكبر من خيانة الدين , الوطن , الهوية , العقيدة , التاريخ .. عليسة المرأة والتي لا تمتلك مما فات شيئا قالت النار ولا العار يا أمة المليار .. #صفقة_القرن #القدس_عاصمة_فلسطين_الأبدية",No,189,صفقة_القرن القدس_عاصمة_فلسطين_الأبدية,,100+,0,2,0.0,4
1,1222030473385345024,صباح الخير على فلسطين وقدسها وشعبها ومهجرينها في المخيمات وأسراها ومبعديها ، صباح الخير لكل من يحمل في قلبه حبها وانتمائها #القدس_عاصمة_فلسطين_الأبديه #تسقط_صفقة_الوهم,No,169,القدس_عاصمة_فلسطين_الأبديه تسقط_صفقة_الوهم,,100+,0,2,0.0,1
2,1222035929105338368,ذنبگ أنگ جميله گ يوسف وخانگ العالم گ أاخواته صباحكم فلسطيني صباح الوطن الجميل #صفقة_القرن_لن_تمر,No,98,صفقة_القرن_لن_تمر,,91-100,0,1,0.0,1
3,1222048121145962496,لا يلزمك ان تكون فلسطينياً لتحب فلسطين، حُبها لارف جنسية أو هوية.. حُبها يكون بالفطرة.. شيء يشبه حُبنا لأمهاتنا بلا تفكير! #صباح_الخير #القدس_عاصمة_فلسطين_الأبدية,No,168,صباح_الخير القدس_عاصمة_فلسطين_الأبدية,,100+,0,2,0.0,4


### Preprocessing dev and dev_test and saving

In [17]:
import csv

def preprocess_dev_data(filename):
    df = load(filename)
    preprocessing(df)
    add_additional_features(df)
    return df

def save_processed_dev_data(df, filepath):
    df.to_csv(filepath, sep='\t', index=False, quoting=csv.QUOTE_NONE)

#'CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev.tsv'
dev_df = preprocess_dev_data('CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev.tsv')
#'CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev-test.tsv'
dev_test_df = preprocess_dev_data('CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev-test.tsv')

save_processed_dev_data(dev_df, 'processed_arabic_dev.tsv')
save_processed_dev_data(dev_test_df, 'processed_arabic_dev_test.tsv')
save_processed_dev_data(train_df, 'processed_arabic_train.tsv')


(1093, 4)
Class Distribution:
class_label
No     62.397072
Yes    37.602928
Name: proportion, dtype: float64
Missing values:
tweet_id       0
tweet_url      0
tweet_text     0
class_label    0
text_length    0
dtype: int64
Number of duplicate rows: 0
Number of Arabic tweets: 1093
Number of tweets with URL: 699
Number of tweets with noise: 741
Number of tweets with non-ASCII characters: 28

Examples:
1      ما الذي تريده قطر من القمة؟ https://t.co/UO9doaeGnY تريد فتح الاجواء الجوية لتضمن اقامة كأس العالم تعتقد إن حدث هام ككأس العالم ربما تستطيع أن تغسل به سمعتها السيئة، تبيض بها سجلها الإجرامي، لذلك قدمت هذا التنازل «الصمت الإعلامي» مقابل فتح الأجواء الجوية... فهل الموضوع يستحق؟!
423                         وصل سعرها إلى 100 ألف دولار.. عرض منصة #الكونجرس الشهيرة، المسروقة خلال اقتحامه من قبَل أنصار #ترامب، على منصة البيع الإلكترونية «eBay» بسعر بدأ من 15 ألف دولار، واختفاء الإعلان بعدما كُتب على الموقع أنها بيعت مقابل 99.900 ألف دولار https://t.co/pk37Cs10im
470                        